In [1]:
import cv2
import dlib
import os
import time
import numpy as np

def resize_image(image: np.ndarray, max_size: int = 800) -> np.ndarray:
    """
    Resize the image while maintaining aspect ratio, so that the maximum dimension is not greater than max_size.
    
    Args:
        - image (numpy.ndarray): Input image.
        - max_size (int): Maximum size for the larger dimension after resizing. Default is 800.

    Returns:
        numpy.ndarray: Resized image.
    """
    if image is None:
        return None
    
    height, width = image.shape[:2]
    if max(height, width) > max_size:
        if height > width:
            new_height = max_size
            new_width = int(width * (max_size / height))
        else:
            new_width = max_size
            new_height = int(height * (max_size / width))
        image = cv2.resize(image, (new_width, new_height))
    return image

def dlib_cnn_detect_faces(source_folder: str, destination_folder: str, model_path: str) -> None:
    """
    Detect faces using DLIB's CNN face detector and save bounding boxes to text files.
    
    Args:
        - source_folder (str): Path to the source directory containing image files.
        - destination_folder (str): Path to the destination directory where processed images and labels will be saved.
        - model_path (str): Path to the pre-trained model for face detection.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Ensure the destination folders exist
    images_folder = os.path.join(destination_folder, "images")
    labels_folder = os.path.join(destination_folder, "labels")
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    if not os.path.exists(labels_folder):
        os.makedirs(labels_folder)
    
    # Load the CNN face detector
    cnn_detector = dlib.cnn_face_detection_model_v1(model_path)
    
    # Iterate through all files in the source directory
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')) and not filename.startswith('.'):
            file_path = os.path.join(source_folder, filename)
            # Load the image
            # Skip files that are not regular image files
            try:
                image = cv2.imread(file_path)
            except:
                print(f"Skipping file: {file_path}")
                continue
            
            # Resize the image while maintaining aspect ratio
            image = resize_image(image)
            
            # Record start time
            start = time.time()
            
            # Perform face detection
            detections = cnn_detector(image, 2)
            
            for i, face in enumerate(detections):
                l, t, r, b = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()
                
                # Ensure x1, y1, x2, y2 format
                x1, y1, x2, y2 = min(l, r), min(t, b), max(l, r), max(t, b)
                
                # Draw bounding boxes for each detected face
                cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                
                # Save bounding box coordinates to a text file
                label_file_path = os.path.join(labels_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(label_file_path, 'w') as label_file:
                    label_file.write(f"{x1} {y1} {x2} {y2}\n")
            
            # Record end time
            end = time.time()
            
            print(f"Processed {filename} in {end - start:.2f} seconds.")
            
            # Save the processed image to the images folder
            output_image_path = os.path.join(images_folder, filename)
            cv2.imwrite(output_image_path, image)
    print("Processing complete.")


In [2]:
def dlib_cnn_detect_faces_score(source_folder: str, destination_folder: str, model_path: str) -> None:
    """
    Detect faces using DLIB's CNN face detector and save bounding boxes to text files with confidence scores.
    
    Args:
        - source_folder (str): Path to the source directory containing image files.
        - destination_folder (str): Path to the destination directory where processed images and labels will be saved.
        - model_path (str): Path to the pre-trained model for face detection.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    images_folder = os.path.join(destination_folder, "images")
    labels_folder = os.path.join(destination_folder, "labels")
    os.makedirs(images_folder, exist_ok=True)
    os.makedirs(labels_folder, exist_ok=True)
    
    cnn_detector = dlib.cnn_face_detection_model_v1(model_path)
    
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')) and not filename.startswith('.'):
            file_path = os.path.join(source_folder, filename)
            try:
                image = cv2.imread(file_path)
            except:
                print(f"Skipping file: {file_path}")
                continue

            image = resize_image(image)
            start = time.time()
            detections = cnn_detector(image, 2)

            label_file_path = os.path.join(labels_folder, f"{os.path.splitext(filename)[0]}.txt")
            with open(label_file_path, 'a') as label_file:  # Use 'a' to append if processing multiple faces
                for i, face in enumerate(detections):
                    l, t, r, b = face.rect.left(), face.rect.top(), face.rect.right(), face.rect.bottom()
                    confidence = face.confidence
                    
                    x1, y1, x2, y2 = min(l, r), min(t, b), max(l, r), max(t, b)
                    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    
                    # Include confidence value with bounding box coordinates
                    label_file.write(f"{confidence} {x1} {y1} {x2} {y2}\n")

            end = time.time()
            print(f"Processed {filename} in {end - start:.2f} seconds.")
            
            output_image_path = os.path.join(images_folder, filename)
            cv2.imwrite(output_image_path, image)
    print("Processing complete.")

In [11]:
source_folder = "/Users/jocareher/Downloads/benchmark_test_split_rotated/4/images"
destination_folder = "/Users/jocareher/Downloads/dlib_predictions_new/4"
model_path = "/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/eLearning/Face_Detection/Weights/mmod_human_face_detector.dat"
dlib_cnn_detect_faces(source_folder=source_folder,
                      destination_folder=destination_folder,
                      model_path=model_path)

Processed flip_face_img_8644.jpg in 9.44 seconds.
Processed flip_face_img_5017.jpg in 4.84 seconds.
Processed flip_face_img_3314.jpg in 5.45 seconds.
Processed flip_face_img_1073.jpg in 4.21 seconds.
Processed face_bcn_507.jpg in 7.62 seconds.
Processed face_bcn_249.jpg in 8.65 seconds.
Processed face_img_5313.jpg in 3.62 seconds.
Processed face_img_2118.jpg in 5.95 seconds.
Processed face_img_1388.jpg in 3.80 seconds.
Processed flip_face_img_3470.jpg in 4.52 seconds.
Processed flip_face_img_7549.jpg in 6.13 seconds.
Processed flip_face_bcn_540.jpg in 5.76 seconds.
Processed flip_face_bcn_187.jpg in 6.59 seconds.
Processed flip_face_img_6040.jpg in 4.22 seconds.
Processed face_img_4431.jpg in 6.27 seconds.
Processed face_img_7660.jpg in 3.16 seconds.
Processed face_img_6998.jpg in 3.13 seconds.
Processed face_img_2320.jpg in 6.66 seconds.
Processed face_bcn_307.jpg in 5.11 seconds.
Processed face_img_8369.jpg in 3.38 seconds.
Processed face_img_6393.jpg in 5.80 seconds.
Processed face_

In [8]:
source_folder ="/Users/jocareher/Downloads/benchmark_test_split_rotated/1/images"
destination_folder = "/Users/jocareher/Downloads/dlib_predictions_score/1"
model_path = "/Users/jocareher/Library/CloudStorage/OneDrive-Personal/Educación/eLearning/Face_Detection/Weights/mmod_human_face_detector.dat"
dlib_cnn_detect_faces_score(source_folder=source_folder,
                      destination_folder=destination_folder,
                      model_path=model_path)

Processed flip_face_img_8491.jpg in 4.36 seconds.
Processed flip_face_img_6866.jpg in 4.98 seconds.
Processed face_img_1957.jpg in 3.38 seconds.
Processed flip_face_img_4730.jpg in 2.96 seconds.
Processed face_bcn_510.jpg in 6.31 seconds.
Processed flip_face_img_5406.jpg in 3.18 seconds.
Processed flip_face_img_4888.jpg in 4.61 seconds.
Processed face_img_8191.jpg in 4.92 seconds.
Processed face_img_4218.jpg in 6.08 seconds.
Processed flip_face_img_8283.jpg in 3.55 seconds.


KeyboardInterrupt: 